Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [21]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression

In [18]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    with tf.device("/cpu:0"):
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        # Variables.
        weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(tf_train_dataset, weights) + biases
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        
        regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
        loss += 5e-4 * regularizers

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.884193
Minibatch accuracy: 21.1%
Validation accuracy: 15.5%
Test accuracy: 16.1%
Minibatch loss at step 500: 2.601035
Minibatch accuracy: 70.3%
Validation accuracy: 76.4%
Test accuracy: 82.4%
Minibatch loss at step 1000: 2.177069
Minibatch accuracy: 70.3%
Validation accuracy: 77.9%
Test accuracy: 84.6%
Minibatch loss at step 1500: 1.438796
Minibatch accuracy: 79.7%
Validation accuracy: 79.1%
Test accuracy: 84.9%
Minibatch loss at step 2000: 1.242004
Minibatch accuracy: 76.6%
Validation accuracy: 80.2%
Test accuracy: 86.3%
Minibatch loss at step 2500: 1.194675
Minibatch accuracy: 75.8%
Validation accuracy: 80.9%
Test accuracy: 87.2%
Minibatch loss at step 3000: 0.814204
Minibatch accuracy: 84.4%
Validation accuracy: 81.4%
Test accuracy: 87.2%


### Neural Network

In [40]:
batch_size = 128
hidden1_units = 1024

graph = tf.Graph()
with graph.as_default():
    with tf.device("/cpu:0"):
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        images = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
        labels = tf.placeholder(tf.float32, shape=(None, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        with tf.name_scope('hidden1'):
            weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units], 
                                                      stddev=1.0 / math.sqrt(float(image_size * image_size)), 
                                                                             name='weights'))
            biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
            regularizers_hidden1 = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))

        with tf.name_scope('logits'):
            weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels], 
                                                      stddev=1.0 / math.sqrt(float(hidden1_units)), name='weights'))
            biases = tf.Variable(tf.zeros([num_labels]), name='biases')
            logits = tf.matmul(hidden1, weights) + biases
            regularizers_logits = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))

        # Training computation.
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        
        regularizers = (regularizers_hidden1 + regularizers_logits)
        loss += 5e-6 * regularizers

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        prediction = tf.nn.softmax(logits)

In [31]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {images : batch_data, labels : batch_labels}
    _, l, train_predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_valid_dataset.eval()}), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_test_dataset.eval()}), test_labels))

Initialized
Minibatch loss at step 0: 2.379452
Minibatch accuracy: 3.9%
Validation accuracy: 29.9%
Test accuracy: 32.6%
Minibatch loss at step 500: 0.739410
Minibatch accuracy: 77.3%
Validation accuracy: 85.8%
Test accuracy: 91.3%
Minibatch loss at step 1000: 0.494735
Minibatch accuracy: 85.2%
Validation accuracy: 87.9%
Test accuracy: 93.2%
Minibatch loss at step 1500: 0.401314
Minibatch accuracy: 88.3%
Validation accuracy: 88.0%
Test accuracy: 93.4%
Minibatch loss at step 2000: 0.655914
Minibatch accuracy: 84.4%
Validation accuracy: 87.6%
Test accuracy: 92.9%
Minibatch loss at step 2500: 0.516454
Minibatch accuracy: 84.4%
Validation accuracy: 89.1%
Test accuracy: 94.0%
Minibatch loss at step 3000: 0.293022
Minibatch accuracy: 91.4%
Validation accuracy: 88.9%
Test accuracy: 94.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [42]:
num_steps = 3001

train_dataset_small = train_dataset[:10000,:]
train_labels_small = train_labels[:10000,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {images : batch_data, labels : batch_labels}
    _, l, train_predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_valid_dataset.eval()}), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_test_dataset.eval()}), test_labels))

Initialized
Minibatch loss at step 0: 2.298042
Minibatch accuracy: 7.0%
Validation accuracy: 30.2%
Test accuracy: 32.8%
Minibatch loss at step 500: 0.155368
Minibatch accuracy: 95.3%
Validation accuracy: 84.6%
Test accuracy: 90.2%
Minibatch loss at step 1000: 0.084285
Minibatch accuracy: 98.4%
Validation accuracy: 84.2%
Test accuracy: 90.2%
Minibatch loss at step 1500: 0.036447
Minibatch accuracy: 98.4%
Validation accuracy: 84.7%
Test accuracy: 90.7%
Minibatch loss at step 2000: 0.018414
Minibatch accuracy: 100.0%
Validation accuracy: 85.2%
Test accuracy: 90.9%
Minibatch loss at step 2500: 0.008097
Minibatch accuracy: 100.0%
Validation accuracy: 84.9%
Test accuracy: 90.8%
Minibatch loss at step 3000: 0.104875
Minibatch accuracy: 98.4%
Validation accuracy: 85.2%
Test accuracy: 90.9%


The model overfits the training data (100% accuracy) and performs poorly on validation and test sets.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [43]:
batch_size = 128
hidden1_units = 1024

graph_dropout = tf.Graph()
with graph_dropout.as_default():
    with tf.device("/cpu:0"):
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        images = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
        labels = tf.placeholder(tf.float32, shape=(None, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        with tf.name_scope('hidden1'):
            weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units], 
                                                      stddev=1.0 / math.sqrt(float(image_size * image_size)), 
                                                                             name='weights'))
            biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
            regularizers_hidden1 = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
            
            # Add a 50% dropout during training only. Dropout also scales
            # activations such that no rescaling is needed at evaluation time.
            keep_prob = tf.placeholder(tf.float32)
            hidden1 = tf.nn.dropout(hidden1, keep_prob, seed=17)

        with tf.name_scope('logits'):
            weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels], 
                                                      stddev=1.0 / math.sqrt(float(hidden1_units)), name='weights'))
            biases = tf.Variable(tf.zeros([num_labels]), name='biases')
            logits = tf.matmul(hidden1, weights) + biases
            regularizers_logits = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))

        # Training computation.
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        
        regularizers = (regularizers_hidden1 + regularizers_logits)
        loss += 5e-6 * regularizers

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        prediction = tf.nn.softmax(logits)

In [47]:
num_steps = 3001

train_dataset_small = train_dataset[:10000,:]
train_labels_small = train_labels[:10000,:]

with tf.Session(graph=graph_dropout) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {images : batch_data, labels : batch_labels, keep_prob: 0.2}
    _, l, train_predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_valid_dataset.eval(), 
                                                                                keep_prob: 1.0}), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_test_dataset.eval(), 
                                                                          keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.428704
Minibatch accuracy: 12.5%
Validation accuracy: 25.7%
Test accuracy: 27.7%
Minibatch loss at step 500: 0.598036
Minibatch accuracy: 82.8%
Validation accuracy: 83.8%
Test accuracy: 89.3%
Minibatch loss at step 1000: 1.059563
Minibatch accuracy: 73.4%
Validation accuracy: 83.7%
Test accuracy: 89.4%
Minibatch loss at step 1500: 0.600776
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Test accuracy: 90.0%
Minibatch loss at step 2000: 0.745777
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Test accuracy: 89.7%
Minibatch loss at step 2500: 0.751767
Minibatch accuracy: 82.0%
Validation accuracy: 84.3%
Test accuracy: 90.1%
Minibatch loss at step 3000: 0.895883
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Test accuracy: 90.0%


A dropout with keep_prob = 0.2 seems to avoid overfitting on training data. Training and validation scores are now similar.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [93]:
batch_size = 128
hidden1_units = 512
hidden2_units = 512
hidden3_units = 256

graph_multi = tf.Graph()
with graph_multi.as_default():
    with tf.device("/cpu:0"):
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        images = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
        labels = tf.placeholder(tf.float32, shape=(None, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        with tf.name_scope('hidden1'):
            weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units], 
                                                      stddev=1.0 / math.sqrt(float(image_size * image_size)), 
                                                                             name='weights'))
            biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
            regularizers_hidden1 = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
            
            # Add a 50% dropout during training only. Dropout also scales
            # activations such that no rescaling is needed at evaluation time.
            keep_prob = tf.placeholder(tf.float32)
            #hidden1 = tf.nn.dropout(hidden1, keep_prob, seed=17)
            
        with tf.name_scope('hidden2'):
            weights = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units], 
                                                      stddev=1.0 / math.sqrt(float(hidden1_units)), 
                                                                             name='weights'))
            biases = tf.Variable(tf.zeros([hidden2_units]), name='biases')
            hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
            regularizers_hidden2 = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
            
            # Add a 50% dropout during training only. Dropout also scales
            # activations such that no rescaling is needed at evaluation time.
            hidden2 = tf.nn.dropout(hidden2, keep_prob, seed=17)
            
        '''with tf.name_scope('hidden3'):
            weights = tf.Variable(tf.truncated_normal([hidden2_units, hidden3_units], 
                                                      stddev=1.0 / math.sqrt(float(hidden2_units)), 
                                                                             name='weights'))
            biases = tf.Variable(tf.zeros([hidden3_units]), name='biases')
            hidden3 = tf.nn.relu(tf.matmul(hidden2, weights) + biases)
            regularizers_hidden3 = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
            
            # Add a 50% dropout during training only. Dropout also scales
            # activations such that no rescaling is needed at evaluation time.
            #keep_prob = tf.placeholder(tf.float32)
            hidden3 = tf.nn.dropout(hidden3, keep_prob, seed=17)
'''
        with tf.name_scope('logits'):
            weights = tf.Variable(tf.truncated_normal([hidden2_units, num_labels], 
                                                      stddev=1.0 / math.sqrt(float(hidden2_units)), name='weights'))
            biases = tf.Variable(tf.zeros([num_labels]), name='biases')
            logits = tf.matmul(hidden2, weights) + biases
            regularizers_logits = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))

        # Training computation.
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        
        regularizers = (regularizers_hidden1 + regularizers_hidden2 + 
                        regularizers_logits)
        loss += 5e-5 * regularizers

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
        
        #optimizer = tf.train.AdamOptimizer(1e-2).minimize(loss)
        
        # Optimizer: set up a variable that's incremented once per batch and
        # controls the learning rate decay.
        '''batch = tf.Variable(0, dtype=tf.int32)
        learning_rate = tf.train.exponential_decay(
          0.01,                # Base learning rate.
          batch * batch_size,  # Current index into the dataset.
          train_labels.shape[0],          # Decay step.
          0.95,                # Decay rate.
          staircase=True)
        
        # Use simple momentum for the optimization.
        optimizer = tf.train.MomentumOptimizer(learning_rate,
                                             0.9).minimize(loss,
                                                           global_step=batch)'''

        # Predictions for the training, validation, and test data.
        prediction = tf.nn.softmax(logits)

In [94]:
num_steps = 10001

with tf.Session(graph=graph_multi) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {images : batch_data, labels : batch_labels, keep_prob: 0.5}
    _, l, train_predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_valid_dataset.eval(), 
                                                                                keep_prob: 1.0}), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict={images: tf_test_dataset.eval(), 
                                                                          keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.347546
Minibatch accuracy: 7.0%
Validation accuracy: 17.7%
Test accuracy: 19.0%
Minibatch loss at step 500: 0.729292
Minibatch accuracy: 78.1%
Validation accuracy: 84.6%
Test accuracy: 90.1%
Minibatch loss at step 1000: 0.706238
Minibatch accuracy: 76.6%
Validation accuracy: 85.7%
Test accuracy: 91.3%
Minibatch loss at step 1500: 0.512541
Minibatch accuracy: 86.7%
Validation accuracy: 86.8%
Test accuracy: 92.1%
Minibatch loss at step 2000: 0.594374
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Test accuracy: 92.4%
Minibatch loss at step 2500: 0.579188
Minibatch accuracy: 83.6%
Validation accuracy: 87.7%
Test accuracy: 93.1%
Minibatch loss at step 3000: 0.352088
Minibatch accuracy: 89.8%
Validation accuracy: 88.2%
Test accuracy: 93.4%
Minibatch loss at step 3500: 0.423985
Minibatch accuracy: 92.2%
Validation accuracy: 88.6%
Test accuracy: 93.6%
Minibatch loss at step 4000: 0.460233
Minibatch accuracy: 85.9%
Validation accuracy: 88.9%
Test a